# Starter Notebook

Install and import required libraries

In [ ]:
!pip install transformers datasets evaluate accelerate peft trl bitsandbytes
!pip install nvidia-ml-py3

In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from transformers import (
    RobertaForSequenceClassification,
    RobertaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType
)
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

## Load Tokenizer and Preprocess Data

In [ ]:
# Constants
BASE_MODEL = "roberta-base"
DATASET_NAME = "ag_news"
OUTPUT_DIR = "roberta_lora_agnews_results"
MAX_LENGTH = 128
RANDOM_SEED = 37

# Load Dataset
dataset = load_dataset(DATASET_NAME, split='train')
print(f"Loaded {len(dataset)} training samples.")

# Load Tokenizer
print(f"Loading tokenizer for base model: {BASE_MODEL}")
tokenizer = RobertaTokenizer.from_pretrained(BASE_MODEL)

# Preprocessing Function
def preprocess(examples):
    """Tokenizes the text data with consistent parameters."""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors=None # for batch processing
    )

# Apply Preprocessing
tokenized_dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=["text"]
    )
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# Explicit label mapping ensures consistency between training and inference
id2label = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}
label2id = {v: k for k, v in id2label.items()}

print("Explicit label mapping for model:")
print(f"id2label: {id2label}")
print(f"label2id: {label2id}")

In [ ]:
# Extract the number of classess and their names
num_labels = dataset.features['label'].num_classes
class_names = dataset.features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}
label2id = {label: i for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

## Load Pre-trained Model
Set up config for pretrained model and download it from hugging face

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    BASE_MODEL,
    num_labels=4,
    id2label=id2label,
    label2id=label2id)
model

## Anything from here on can be modified

In [ ]:
# Split Dataset
split_datasets = tokenized_dataset.train_test_split(
    test_size=0.1,
    seed=37,
    stratify_by_column="labels"
    )
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]
print(f"Training set size: {len(train_dataset)}")
print(f"Evaluation set size: {len(eval_dataset)}")

In [ ]:
# Freeze all parameters of the base model
for param in model.parameters():
    param.requires_grad = False

## Setup LoRA Config
Setup PEFT config and get peft model for finetuning

In [ ]:
# Running PEFT Config on LoRA configuration
peft_config = LoraConfig(
    r=8, # Rank of the update matrices, higher it is, better the capacity
    lora_alpha=16, # LoRA scaling factor = 2*Rank
    lora_dropout=0.2, # Dropout probability for LoRA layers
    bias = 'lora_only',
    target_modules = ["query", "value"],
    task_type="SEQ_CLS",
)

In [ ]:
print("Applying LoRA configuration to the base model-")
peft_model = get_peft_model(model, peft_config)
peft_model

In [ ]:
print('PEFT Model')
peft_model.print_trainable_parameters()

In [ ]:
# Calculate trainable parameters manually for verification
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in peft_model.parameters())
print(f"Trainable parameters: {trainable_params:,}")
print(f"Total parameters: {total_params:,}")
print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.4f}%")

# Verify we're under 1M trainable parameters
assert trainable_params < 1_000_000, "Trainable parameters exceed 1 million!"

In [ ]:
print("Trainable parameters:")
for name, param in peft_model.named_parameters():
    if param.requires_grad:
         print(name)

## Training Setup

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# To track evaluation accuracy during training
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [ ]:
# Setup Training args
output_dir = "results"
training_args = TrainingArguments(
    output_dir="roberta-lora-agnews-results",
    # learning_rate=5e-4,
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=6,
    # max_steps=1200 # consider either max steps or desired epochs
    eval_strategy='steps', # Evaluate during training
    eval_steps=200,
    save_strategy="steps", # Save checkpoints during training
    save_steps=200, # Save checkpoints at the same frequency as evaluation
    load_best_model_at_end=True, # Load the best model found during training
    metric_for_best_model="accuracy", # Use accuracy to determine the best model
    greater_is_better=True,
    weight_decay=0.01,
    optim="adamw_torch",
    max_grad_norm=1.0,
    lr_scheduler_type="linear",
    report_to=None, # Disable default reporting integrations like wandb/tensorboard unless configured
    fp16=torch.cuda.is_available(), # Use mixed precision if GPU is available
    gradient_accumulation_steps=4, # Accumulate gradients to simulate larger batch size if needed
    warmup_ratio=0.1, # Add a learning rate warmup phase
    seed=37,
    logging_steps=50,
    gradient_checkpointing=True, # Enable if memory is constrained, might slow down training
)

print("\nTraining Arguments:")
print(training_args)

In [ ]:
# initialize trainer
def get_trainer(model):
      return  Trainer(
          model=peft_model,
          args=training_args,
          compute_metrics=compute_metrics,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
          tokenizer=tokenizer,
      )

### Start Training

In [ ]:
peft_lora_finetuning_trainer = get_trainer(peft_model)

result = peft_lora_finetuning_trainer.train()

In [ ]:
# Save Final Model
print("Training finished. Saving model...")
peft_lora_finetuning_trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))

## Evaluate Finetuned Model


### Performing Inference on Custom Input
Uncomment following functions for running inference on custom inputs

In [ ]:
# Evaluate Model
eval_metrics = peft_lora_finetuning_trainer.evaluate()
print(f"Final evaluation metrics: {eval_metrics}")

In [ ]:
def classify(model, tokenizer, text):
    """Run inference on a single text example."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.to(device)
    model.eval()
    with torch.no_grad():
        output = model(**inputs)
    prediction = output.logits.argmax(dim=-1).item()
    print(f'Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
    return id2label[prediction]

### Run Inference on eval_dataset

In [ ]:
# Run Inference on Test Examples
test_examples = [
    "Wall Street rallies as tech stocks surge to new heights",
    "Manchester United wins dramatic match against Liverpool",
    "New study reveals breakthrough in cancer treatment",
    "Tech company announces innovative AI product",
    "NVIDIA owns AI"
]

print("\nRunning inference on test examples:")
for text in test_examples:
    result = classify(peft_model, tokenizer, text)

In [ ]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

def evaluate_model(inference_model, dataset, labelled=True, batch_size=32, data_collator=None):
    """
    Evaluate a PEFT model on a dataset.

    Args:
        inference_model: The model to evaluate.
        dataset: The dataset (Hugging Face Dataset) to run inference on.
        labelled (bool): If True, the dataset includes labels and metrics will be computed.
                         If False, only predictions will be returned.
        batch_size (int): Batch size for inference.
        data_collator: Function to collate batches. If None, the default collate_fn is used.

    Returns:
        If labelled is True, returns a tuple (metrics, predictions)
        If labelled is False, returns the predictions.
    """
    # Create the DataLoader
    eval_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=data_collator)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    all_labels = []

    with torch.no_grad():
      # Loop over the DataLoader
      for batch in tqdm(eval_dataloader, desc="Evaluating"):
          # Move each tensor in the batch to the device
          batch = {k: v.to(device) for k, v in batch.items()}

          # Forward pass
          outputs = inference_model(**batch)
          predictions = outputs.logits.argmax(dim=-1)

          all_predictions.extend(predictions.cpu().numpy())
          if labelled and "labels" in batch:
              all_labels.extend(batch["labels"].cpu().numpy())
              # Calculate accuracy
              metric = evaluate.load('accuracy')

    # Compute metrics if dataset is labelled
    if labelled and all_labels:
        accuracy = accuracy_score(all_labels, all_predictions)

        metrics = {
            "accuracy": accuracy,
        }

        print(f"Evaluation metrics: {metrics}")
        return metrics, all_predictions

    return all_predictions

In [ ]:
# Process Test Data for Submission
try:
    print("\nLoading test data for Kaggle submission...")
    test_data_path = "test_unlabelled.pkl"

    # Load the pickle file
    test_df = pd.read_pickle(test_data_path)
    print(f"Loaded data type: {type(test_df)}")

    # If it's already a Dataset object, convert to DataFrame first
    if hasattr(test_df, 'to_pandas'):
        print("Converting Dataset to DataFrame...")
        test_df = test_df.to_pandas()
    elif not isinstance(test_df, pd.DataFrame):
        print(f"Unexpected data type: {type(test_df)}. Converting to DataFrame...")
        if hasattr(test_df, 'features'):
            # It's a Dataset-like object
            test_df = pd.DataFrame({col: test_df[col] for col in test_df.column_names})
        else:
            # Try to convert directly
            test_df = pd.DataFrame(test_df)

    print(f"Successfully loaded test data with shape: {test_df.shape}")

    # Ensure 'ID' column exists in the DataFrame
    if "ID" not in test_df.columns:
        test_df["ID"] = range(len(test_df))

    # Convert to Hugging Face Dataset
    test_dataset = Dataset.from_pandas(test_df)

    # Apply the same preprocessing as training data
    test_dataset = test_dataset.map(preprocess, batched=True, remove_columns=["text"])

    # Run inference
    print("Running inference on test data...")
    predictions = peft_lora_finetuning_trainer.predict(test_dataset)
    pred_labels = np.argmax(predictions.predictions, axis=-1)

    # Create submission DataFrame with EXACT column names required
    submission_df = pd.DataFrame({
        'ID': test_df["ID"],  # Use the original IDs from the test set!
        'Label': pred_labels  # Numeric labels (0, 1, 2, 3)
    })

    # Save to CSV without index
    submission_path = os.path.join(OUTPUT_DIR, "kaggle_submission.csv")
    submission_df.to_csv(submission_path, index=False)
    print(f"Kaggle submission file saved to {submission_path}")
    print(submission_df.head())

except Exception as e:
    print(f"Error processing test data: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
print(f"Base Model: {BASE_MODEL}")
print(f"LoRA Rank (r): {peft_config.r}")
print(f"LoRA Alpha: {peft_config.lora_alpha}")
print(f"Target Modules: {peft_config.target_modules}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Final Evaluation Accuracy: {eval_metrics['eval_accuracy']:.4f}")

In [ ]:
import json
import matplotlib.pyplot as plt
def extract_metrics_from_logs(output_dir):
    log_file = os.path.join(output_dir, "trainer_state.json")
    if not os.path.exists(log_file):
        print(f"Log file not found at {log_file}")
        return None

    with open(log_file, 'r') as f:
        logs = json.load(f)

    metrics_dict = {
        'train_loss': [],
        'eval_loss': [],
        'eval_accuracy': [],
        'steps': []
    }

    for log in logs.get('log_history', []):
        if 'loss' in log and 'step' in log and 'eval_loss' not in log:
            metrics_dict['train_loss'].append(log['loss'])
            metrics_dict['steps'].append(log['step'])

        if 'eval_loss' in log and 'eval_accuracy' in log:
            metrics_dict['eval_loss'].append(log['eval_loss'])
            metrics_dict['eval_accuracy'].append(log['eval_accuracy'])

    return metrics_dict

# 1. Plot training and evaluation metrics over time
def plot_training_metrics(metrics_dict):
    plt.figure(figsize=(12, 8))

    # Plot training loss
    plt.subplot(2, 1, 1)
    plt.plot(metrics_dict['steps'][:len(metrics_dict['train_loss'])],
             metrics_dict['train_loss'], 'b-', label='Training Loss')

    # Create positions for eval metrics
    eval_steps = np.linspace(0, max(metrics_dict['steps']), len(metrics_dict['eval_loss']))
    plt.plot(eval_steps, metrics_dict['eval_loss'], 'r-', label='Evaluation Loss')

    plt.title('Loss vs Training Steps')
    plt.xlabel('Training Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Plot evaluation accuracy
    plt.subplot(2, 1, 2)
    plt.plot(eval_steps, metrics_dict['eval_accuracy'], 'g-', label='Evaluation Accuracy')
    plt.title('Accuracy vs Training Steps')
    plt.xlabel('Training Steps')
    plt.ylabel('Accuracy')
    plt.ylim(0.8, 1.0)  # Assuming accuracy is between 0.8 and 1.0, adjust as needed
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('training_metrics.png', dpi=300)
    plt.show()

In [ ]:
metrics = extract_metrics_from_logs("roberta-lora-agnews-results/checkpoint-800/")
plot_training_metrics(metrics)